##### Создание датафрейма с данными о фактическом давлении

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
result_data = main()
if result_data:
    print("\n" + "=" * 80)
    print("ДАННЫЕ УСПЕШНО ПОДГОТОВЛЕНЫ ДЛЯ СРАВНЕНИЯ!")
    print("=" * 80)
    print(f"Структура данных содержит:")
    print(f"  - Фактические данные: {len(result_data['fact_data'])} записей")
    print(f"  - Данные по скважинам: {len(result_data['well_dataframes'])} скважин")
    print(f"  - Интерполированные модельные данные: {len(result_data['models_interpolated'])} моделей")
    print(f"  - Всего записей: {result_data['statistics']['total_records']}")
    print(f"  - Параметры: {list(result_data['statistics']['parameter_counts'].keys())}")
    print(f"  - Excel файл: {result_data['excel_file']}")
    
    # Пример доступа к данным
    if result_data['well_dataframes']:
        first_well = list(result_data['well_dataframes'].keys())[0]
        print(f"\nПример данных для скважины {first_well}:")
        print(result_data['well_dataframes'][first_well].head())

In [8]:
def parse_well_data(file_path):
    """
    Парсит файл с данными скважин и возвращает DataFrame
    """
    encodings = ['utf-8', 'cp1251', 'cp1252', 'latin1', 'iso-8859-1', 'windows-1251']

    # Читаем файл
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                lines = f.readlines()
            print(f'Успешно прочитано с кодировкой {encoding}')
            break
        except UnicodeDecodeError:
            continue
    else:
        raise ValueError('Файл не читается в представленных кодировках')
    
    data = []
    
    for line in lines:
        # Пропускаем строки с комментариями и служебной информацией
        if line.startswith('--') or line.startswith('PC_') or 'C:\\' in line:
            continue
        
        # Разделяем строку на элементы
        parts = line.strip().split()
        
        # Проверяем, что строка содержит 3 значения (well_fact, date_fact, wpb_bar_fact)
        if len(parts) == 3:
            try:
                well_fact = str(parts[0])
                date_str = parts[1]
                wpb_bar_fact = float(parts[2])
                
                # Преобразуем дату в формат datetime
                try:
                    date_fact = datetime.strptime(date_str, '%d.%m.%Y')
                except ValueError as exc:
                    print(f'ОШИБКА извлечения даты в строке "{line}"')
                    print(f'ОШИБКА: {str(exc)}')
                    continue
                
                data.append({
                    'well_fact': well_fact,
                    'date_fact': date_fact,
                    'wpb_bar_fact': wpb_bar_fact
                })
            except (ValueError, IndexError) as exc:
                print(f'ОШИБКА извлечения данных из строки "{line}"')
                print(f'ОШИБКА: {str(exc)}')
                continue
    
    # Создаем DataFrame
    df = pd.DataFrame(data)
    
    # Сортируем по well_fact и date_fact для удобства
    if not df.empty:
        df = df.sort_values(['well_fact', 'date_fact']).reset_index(drop=True)
    
    return df


In [9]:
# Укажите путь к вашему файлу
file_path = "data/Pfkt0.inc"  # или полный путь к файлу

# Парсим данные
df = parse_well_data(file_path)

# Выводим информацию о DataFrame
print(f"Размер DataFrame: {df.shape}")
print(f"Уникальные скважины: {df['well_fact'].unique()}")
print(f"Диапазон дат: {df['date_fact'].min()} - {df['date_fact'].max()}")

# Показываем первые строки
print("\nПервые 10 строк:")
print(df.head(10))


Успешно прочитано с кодировкой cp1251
Размер DataFrame: (11413, 3)
Уникальные скважины: ['110' '127' '129' '142' '156' '157' '18' '24' '25' '46' '47' '53' '57'
 '63' '70' '97' '99']
Диапазон дат: 2005-01-10 00:00:00 - 2025-03-21 00:00:00

Первые 10 строк:
  well_fact  date_fact  wpb_bar_fact
0       110 2005-01-11          83.3
1       110 2005-01-20          81.9
2       110 2005-01-31          81.2
3       110 2005-02-10          78.4
4       110 2005-02-21          75.7
5       110 2005-02-28          75.6
6       110 2005-03-10          72.9
7       110 2005-03-20          70.8
8       110 2005-03-31          70.8
9       110 2005-04-11          68.6


##### Извлечение уникальных имен скважин

In [10]:
def get_unique_wells(df):
    """
    Извлекает список уникальных имен скважин из DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame с колонкой 'well_fact', содержащей идентификаторы скважин.
    
    Returns:
    --------
    list
        Отсортированный список уникальных имен скважин.
    
    Examples:
    ---------
    >>> df = pd.DataFrame({
    ...     'well_fact': [18, 24, 18, 25, 24],
    ...     'date_fact': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    ...     'wpb_bar_fact': [85.6, 90.2, 86.1, 92.3, 89.7]
    ... })
    >>> get_unique_wells(df)
    [18, 24, 25]
    """
    if df is None or df.empty:
        print("DataFrame пустой или None")
        return []
    
    if 'well_fact' not in df.columns:
        print("В DataFrame отсутствует колонка 'well_fact'")
        print(f"Доступные колонки: {list(df.columns)}")
        return []
    
    # Извлекаем уникальные значения и сортируем их
    unique_wells = sorted(df['well_fact'].unique().tolist())
    
    return unique_wells

In [11]:
unique_wells_list: list[str] = get_unique_wells(df)
print(unique_wells_list)

['110', '127', '129', '142', '156', '157', '18', '24', '25', '46', '47', '53', '57', '63', '70', '97', '99']
